needed methods:
    -oop the function
    -use db instead of json
    
    -move from testing on GCP
    -switch client credentials to web
stretch:
    -test find_overlapping_free_blocks for redundancy

    -remove events starting and endng at the same time
    -accomodate unsorted busy_periods
    -add functionality to query specifc calendar
    -create event on calendar and send invite
    -make the auth persist
    -followup on the soon events bug with gcal, where it ignores soon or ongoign
    events
done:
    -modularize authentication
    -store creds in json to be accesed later
    -google login
    -package find overlap time into one function
    -use overlapping busy times to find overalp free time (ready method)
    -extend beyonf primary
    -query for given user
    -process overlapping busy time and return sorted
    -find free times given more than one calendar
    -find free times given one calendar
    -standard timezone

In [287]:
from __future__ import print_function
import calendar

from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.errors import HttpError

from google.oauth2.credentials import Credentials
from google.oauth2 import id_token
from google.oauth2.credentials import Credentials

from google.auth.exceptions import GoogleAuthError
from googleapiclient.errors import HttpError
from googleapiclient.discovery import build

from datetime import datetime, timedelta, timezone
import pytz
import os
from typing import List, Optional
import json

In [288]:
def apply_timezone_standard(dt_object, timezone_string):
    """
    Takes in a given datetime object without timezone, adds in local timezone
    and converts the timezone to offset seconds format
    """
    timezone_object = pytz.timezone(timezone_string)
    processed_dt_object = "Nil"

    #check if the dt object already has tz offset or not
    if dt_object.tzinfo is None:
        dt_object_localized = timezone_object.localize(dt_object)
    else:
         dt_object_localized = dt_object

    offset_seconds = dt_object_localized.tzinfo.utcoffset(dt_object_localized).total_seconds()
    offset_seconds
    timezone_in_seconds = timezone(timedelta(seconds = offset_seconds))
    processed_dt_object = dt_object_localized.astimezone(timezone_in_seconds)

    return processed_dt_object

In [289]:
def produce_time_window_bounds(time_zone_str: str, window_days: int) -> dict:
    """
    Takes in a timezone and window_days to return a time window bounds
    starting with now and ending in window_days
    """
    time_zone_object = pytz.timezone(time_zone_str)
    plain_start_time = datetime.now(time_zone_object)
    plain_end_time = plain_start_time + timedelta(days = window_days)

    #apply second offset timezone standard to start_time and end_time
    start_time = apply_timezone_standard(plain_start_time, time_zone_str)
    end_time = apply_timezone_standard(plain_end_time, time_zone_str)

    window_bounds = {"window_start": start_time, "window_end": end_time}
    return window_bounds

In [290]:
def sorted_by_start(periods: list) -> list:
    """
    Returns sorted periods by start time
    """
    periods.sort(key = lambda x: x['start'])
    return periods

In [291]:
def user_busy_query_calendars(primary_only: bool, time_zone_str: str, window_days: int,  creds) -> list:
    """
    Returns busy periods in all user calendars, given their credentials through
    querying gcal api

    Notes:
        - The function queries only the primary user calendar when
        primary_only is True
    """
    window_bounds = produce_time_window_bounds(time_zone_str, window_days)

    # Converting to google time format gcal api doesn't accept timezone in
    # timeMin or timeMax str request
    google_format_start_time = window_bounds["window_start"].strftime('%Y-%m-%dT%H:%M:%S') + 'Z'
    google_format_end_time = window_bounds["window_end"].strftime('%Y-%m-%dT%H:%M:%S') + 'Z'

    # print(f"Window start: {google_format_start_time}")
    # print(f"Window end: {google_format_end_time}")

    #loading the primary calendar by default
    calendar_ids = ["primary"]
    try:
        gcal_service_object = build('calendar', 'v3', credentials = creds)
        calendar_list = gcal_service_object.calendarList().list().execute()
        
        #handling primary_only by retrieving all calendar ids and
        # overwriting calendar_list if primary_only is False
        if not primary_only:
            calendar_list = gcal_service_object.calendarList().list().execute()
            calendar_ids = [calendar_entry['id'] for calendar_entry in calendar_list['items']]

        all_busy_periods = []

        for calendar_id in calendar_ids:            
            request_body = {
                'timeMin': google_format_start_time,
                'timeMax': google_format_end_time,
                'timeZone': time_zone_str,
                'items': [{'id': calendar_id}]}
            freebusy_request = gcal_service_object.freebusy().query(body=request_body)
            freebusy_response = freebusy_request.execute()
            busy_periods = freebusy_response['calendars'][calendar_id]['busy']
            #(request_body)
            
            all_busy_periods.extend(busy_periods)

        return sorted_by_start(all_busy_periods)

    except HttpError as error:
        print(f'An HttpError occurred: {error}')
    except ValueError as error:
        print(f'A ValueError occurred: {error}')
    except KeyError as error:
        print(f'A KeyError occurred: {error}')
    except GoogleAuthError as error:
        print(f'A GoogleAuthError occurred: {error}')

In [292]:
def combine_busy_periods(busy_periods_list: list) -> list:
    """
    Takes in all busy periods for all users and returns merged busy
    periods, handling overlapping periods

    Notes:
        - This function also works if only one busy_periods list is provided
        as it's possible for one user's busy_periods to have overlapping events
        from different calendars that need to be merged
    """

    #adding all busy periods lists into one big list, to be processed
    #later in the loop to handle overlapping periods and produce
    #merged busy beriods
    #creating a copy to break the reference to busy_periods_1
    all_busy_periods = sum(list(busy_periods_list.copy()), [])
    
    #sorting all periods by start time to find overalps between them
    all_busy_periods = sorted_by_start(all_busy_periods)

    combined_periods = []
    current_start = all_busy_periods[0]['start']
    current_end = all_busy_periods[0]['end']

    # Iterate through the sorted busy periods to merge overlapping periods
    for period in all_busy_periods[1:]:
        start = period['start']
        end = period['end']

        # If the start time of the current period is within the ongoing period,
        # update the ongoing period's end time if necessary
        if start <= current_end:
            current_end = max(current_end, end)
        # If the current period doesn't overlap with the ongoing period,
        # add the ongoing period to the result list and start a new ongoing period
        else:
            combined_periods.append({'start': current_start, 'end': current_end})
            current_start = start
            current_end = end

    # Add the last ongoing period to the result list
    combined_periods.append({'start': current_start, 'end': current_end})
    return sorted_by_start(combined_periods)

In [293]:
def find_free_blocks(busy_blocks: list, time_zone_str: str, window_days: int) -> list:
    """
    Finds free time blocks given a list of free periods
    """
    window_bounds = produce_time_window_bounds('Africa/Cairo', 1)
    start_time, end_time = window_bounds["window_start"], window_bounds["window_end"]
    
    # Calculate the free time blocks
    free_periods = []
    previous_block_end = start_time
    for busy_block in busy_blocks:
        busy_block_start = datetime.fromisoformat(busy_block['start'])
        busy_block_end = datetime.fromisoformat(busy_block['end'])
        
        if previous_block_end < busy_block_start:
            free_periods.append((previous_block_end, busy_block_start))
        previous_block_end = busy_block_end
    if previous_block_end < end_time:
        free_periods.append((previous_block_end, end_time))
    
    return free_periods

In [294]:
class User:
    """
    User details for interaction with gcal API
    """
    def __init__(self, primary_only, time_zone_str, window_days, credentials) -> None:
        self.primary_only = primary_only
        self.time_zone_str = time_zone_str
        self.window_days = window_days
        self.credentials = credentials

In [295]:
def find_overlapping_free_blocks(viewer_user: User, users_list: list):
    """
    Finds all overlapping free time blocks between all users_list

    """
    busy_separated_users_blocks = []

    for user in users_list:
        busy_user_blocks = user_busy_query_calendars(user.primary_only,
                                                  user.time_zone_str,
                                                  user.window_days,
                                                  user.credentials)
        busy_separated_users_blocks.append(busy_user_blocks)

    #merge all busy user blocks, including overlapping blocks
    busy_combined_users_combo = combine_busy_periods(busy_separated_users_blocks)
    #find all overlapping time blocks and display them with the timezone
    #and preferences of the viewer user
    free_overlapping_blocks = find_free_blocks(busy_combined_users_combo,
                                               viewer_user.time_zone_str,
                                               viewer_user.window_days)
    return free_overlapping_blocks

In [296]:
def authenticate_user(TOKENS_FILE_PATH):
    """
    Authenticate the user save their token with the user's Google ID in the token file
    at TOKENS_FILE_PATH

    Parameters:
        TOKENS_FILE_PATH (str): The file path for the JSON file where the access tokens are stored.
    """
    SCOPES = ['https://www.googleapis.com/auth/calendar', 'openid']

    # Set up the OAuth 2.0 flow
    flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
    creds = flow.run_local_server(port=0)

    # Get the user's Google ID from the ID token
    id_info = id_token.verify_oauth2_token(creds.id_token, Request())
    user_google_id = id_info['sub']

    # Save the token using the user's Google ID
    tokens = {}
    if os.path.exists(TOKENS_FILE_PATH):
        with open(TOKENS_FILE_PATH, 'r') as f:
            tokens = json.load(f)

    tokens[user_google_id] = json.loads(creds.to_json())

    with open(TOKENS_FILE_PATH, 'w') as f:
        json.dump(tokens, f)

    print(f"Token saved for user ID: {user_google_id}")

In [254]:
def get_credentials_by_google_id(user_google_id, TOKENS_FILE_PATH):
    """
    Retrieve the user's credentials from the JSON file using their Google ID.

    Parameters:
        user_google_id (str): The Google ID of the user for whom the credentials are being retrieved.
        TOKENS_FILE_PATH (str): The file path for the JSON file where the access tokens are stored.

    Returns:
        google.oauth2.credentials.Credentials or None: A Credentials object containing the user's credentials if found, otherwise None.
    """
    with open(TOKENS_FILE_PATH, 'r') as f:
        tokens = json.load(f)

    user_credentials = tokens.get(user_google_id)

    if user_credentials:
        return Credentials.from_authorized_user_info(user_credentials)
    else:
        print(f"No credentials found for user ID: {user_google_id}")
        return None

In [297]:
TOKENS_FILE_PATH = "tokens_by_google_id.json"
authenticate_user(TOKENS_FILE_PATH)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=782814832589-2clu5okvv4ms5u2jafaiu2lsb96eio9v.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A59112%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar+openid&state=RYm5IBKvg7hV6ySr9zxRhzVJZG3Y4l&access_type=offline
Token saved for user ID: 117540421435182693557


In [298]:
#primary calendars only

c1 = get_credentials_by_google_id("117540421435182693557", TOKENS_FILE_PATH)
c2 = get_credentials_by_google_id("104932078796484237575", TOKENS_FILE_PATH)

u1 = User(primary_only = True,
          time_zone_str = 'Africa/Cairo',
          window_days = 1,
          credentials = c1)

u2 = User(primary_only = True,
          time_zone_str = 'Africa/Cairo',
          window_days = 1,
          credentials = c2)

find_overlapping_free_blocks(viewer_user = u1, users_list = [u1, u2])

[(datetime.datetime(2023, 4, 20, 22, 36, 45, 618451, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))),
  datetime.datetime(2023, 4, 21, 1, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))),
 (datetime.datetime(2023, 4, 21, 2, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))),
  datetime.datetime(2023, 4, 21, 4, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))),
 (datetime.datetime(2023, 4, 21, 6, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))),
  datetime.datetime(2023, 4, 21, 7, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))),
 (datetime.datetime(2023, 4, 21, 10, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))),
  datetime.datetime(2023, 4, 21, 13, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200)))),
 (datetime.datetime(2023, 4, 21, 13, 30, tzinfo=datetime.timezone(datetime.timedelta(seconds=7200))),
  datetime.datetime(2023, 4, 21, 16, 0, tzinfo=datetime.timezone(datetime.ti